In [ ]:
### PREPARATION ###

import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import macd
from ta.volume import OnBalanceVolumeIndicator
from ta.volatility import AverageTrueRange
from ta.momentum import AwesomeOscillatorIndicator
from ta.momentum import KAMAIndicator
from ta.momentum import PercentagePriceOscillator
from ta.momentum import PercentageVolumeOscillator
from ta.momentum import ROCIndicator
from ta.momentum import StochRSIIndicator
from ta.volume import AccDistIndexIndicator
from ta.volume import ChaikinMoneyFlowIndicator
from ta.volume import EaseOfMovementIndicator
from ta.volume import ForceIndexIndicator
from ta.volume import MFIIndicator
from ta.volume import NegativeVolumeIndexIndicator
from ta.volume import VolumePriceTrendIndicator
from ta.trend import ADXIndicator
from ta.trend import MassIndex
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin



#Loading Datasets

hdfc = pd.read_csv (r'C:\Users\MOHIT.K\Desktop\STock Market Prediction Literature\Stock trend Prediction\HDFC_NSE.csv')
n_50 = pd.read_csv (r'C:\Users\MOHIT.K\Desktop\STock Market Prediction Literature\Stock trend Prediction\NIFTY_50.csv')
n_bank = pd.read_csv (r'C:\Users\MOHIT.K\Desktop\STock Market Prediction Literature\Stock trend Prediction\NIFTY_BANK.csv')

#Renaming Columns and Dropping Date, Volume and Close Columns 

n_50.rename(columns = {'Date':'N50_Date', 'Open':'N50_Open', 'High':'N50_High', 'Low':'N50_Low', 'Close':'N50_Close', 'Adj Close': 'N50_Adj Close', 'Volume': 'N50_Volume'}, inplace = True)
n_bank.rename(columns = {'Date':'NB_Date','Open':'NB_Open', 'High':'NB_High', 'Low':'NB_Low', 'Close':'NB_Close', 'Adj Close': 'NB_Adj Close', 'Volume': 'NB_Volume'}, inplace = True)

HDFC = hdfc.drop(['Adj Close'], axis = 1)
Nifty_50 = n_50.drop(['N50_Date', 'N50_Volume', 'N50_Close', 'N50_Open', 'N50_High', 'N50_Low', 'N50_Adj Close'], axis=1)
Nifty_Bank = n_bank.drop(['NB_Date', 'NB_Volume', 'NB_Close', 'NB_Open', 'NB_High', 'NB_Low', 'NB_Adj Close'], axis=1)

#Merging the datasets and deleting rows with any null values

HDFC_Working = HDFC.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

HDFC_Working.reset_index(drop=True, inplace=True)

### FEATURE EXTRACTION ###

#Calculating Candle Body

HDFC_Working["Candle Body"] = (HDFC_Working["Close"] - HDFC_Working["Open"])

#Calculating Upper and Lower Shadow lengths 

HDFC_Working["Upper Shadow"] = ""
HDFC_Working["Lower Shadow"] = ""

for i in range (len(HDFC_Working)):
    if(HDFC_Working.iat[i, 6] >= 0):
        HDFC_Working.iat[i, 7] = HDFC_Working.iat[i, 2] - HDFC_Working.iat[i, 4]
    else:
        HDFC_Working.iat[i, 7] = HDFC_Working.iat[i, 2] - HDFC_Working.iat[i, 1]
        
for i in range (len(HDFC_Working)):
    if(HDFC_Working.iat[i, 6] >= 0):
        HDFC_Working.iat[i, 8] = HDFC_Working.iat[i, 1] - HDFC_Working.iat[i, 3]
    else:
        HDFC_Working.iat[i, 8] = HDFC_Working.iat[i, 4] - HDFC_Working.iat[i, 3]

#Calculating Range

HDFC_Working["Range"] = ""

for i in range(len(HDFC_Working)):
    HDFC_Working.iat[i, 9] = HDFC_Working.iat[i, 2] - HDFC_Working.iat[i, 3]
                                                                        
#Determining the nature of Candle 

HDFC_Working["Candle Nature"] = ""

for i in range(len(HDFC_Working)):
    if(HDFC_Working.iat[i, 6] > 0):    #Bullish
        HDFC_Working.iat[i, 10] = "Bullish"  
    else:                              #Bearish
        HDFC_Working.iat[i, 10] = "Bearish"
        
#Determining the type of candle

HDFC_Working["Candle Type"] = ""

for i in range(len(HDFC_Working)):   
        
        
    if(0 <= HDFC_Working.iat[i, 7]/HDFC_Working.iat[i, 9] <= 0.15 and 0 <= HDFC_Working.iat[i, 8]/HDFC_Working.iat[i, 9] <= 0.15): #Marubozu
        HDFC_Working.iat[i, 11] = "Marubozu"    

    if(0.1 < abs(HDFC_Working.iat[i, 6])/HDFC_Working.iat[i, 9] <= 0.3 and 0.35<=HDFC_Working.iat[i, 7]/HDFC_Working.iat[i, 9]<=0.45): #Spinning Top
        HDFC_Working.iat[i, 11] = "Spinning Top"
        
    if(0 <= abs(HDFC_Working.iat[i, 6])/HDFC_Working.iat[i, 9] <= 0.05): #Doji
        HDFC_Working.iat[i, 11] = "Doji"
        
    if(HDFC_Working.iat[i, 8] != 0 and 0<=HDFC_Working.iat[i, 7]/HDFC_Working.iat[i, 9]<=0.3 and HDFC_Working.iat[i, 8]/abs(HDFC_Working.iat[i, 6]) >= 2):
        if(HDFC_Working.iat[i, 4] < HDFC_Working.iat[i-1, 4] < HDFC_Working.iat[i-2, 4] < HDFC_Working.iat[i-3, 4]): 
            HDFC_Working.iat[i, 11] = "Hammer"
        if(HDFC_Working.iat[i, 4] > HDFC_Working.iat[i-1, 4] > HDFC_Working.iat[i-2, 4] > HDFC_Working.iat[i-3, 4]):
            HDFC_Working.iat[i, 11] = "Hanging Man"
            
    if(HDFC_Working.iat[i, 7] != 0 and 0<=HDFC_Working.iat[i, 8]/HDFC_Working.iat[i, 9]<=0.3 and HDFC_Working.iat[i, 7]/abs(HDFC_Working.iat[i, 6]) >= 2):
        if(HDFC_Working.iat[i, 4]> HDFC_Working.iat[i-1, 4] > HDFC_Working.iat[i-2, 4] > HDFC_Working.iat[i-3, 4] ): 
            HDFC_Working.iat[i, 11] = "Shooting Star" 
        if(HDFC_Working.iat[i, 4]< HDFC_Working.iat[i-1, 4] < HDFC_Working.iat[i-2, 4] < HDFC_Working.iat[i-3, 4] and HDFC_Working.iat[i-1, 10] == "Bearish" and HDFC_Working.iat[i-2, 10] == "Bearish"): 
            HDFC_Working.iat[i, 11] = "Inverted Hammer"
            

HDFC_Working["CB"] = abs(HDFC_Working["Candle Body"]).copy()

HDFC_Working["EMA"] = HDFC_Working["CB"].ewm(span=25,adjust=False).mean()

for i in range(len(HDFC_Working)):
    
    
    if(i>=25 and HDFC_Working.iat[i, 11] != "Shooting Star" and HDFC_Working.iat[i, 11] != "Inverted Hammer" and HDFC_Working.iat[i, 11] != "Spinning Top" and HDFC_Working.iat[i, 11] != "Doji" and HDFC_Working.iat[i, 11] != "Marubozu" and HDFC_Working.iat[i, 11] != "Hammer" and HDFC_Working.iat[i, 11] != "Hanging Man"):
        if(HDFC_Working.iat[i, 12]/HDFC_Working.iat[i, 13] >= 1):
            HDFC_Working.iat[i, 11] = "Long"
            
    if(i >= 25 and HDFC_Working.iat[i, 11] != "Shooting Star" and HDFC_Working.iat[i, 11] != "Inverted Hammer" and HDFC_Working.iat[i, 11] != "Spinning Top" and HDFC_Working.iat[i, 11] != "Doji" and HDFC_Working.iat[i, 11] != "Marubozu" and HDFC_Working.iat[i, 11] != "Hammer" and HDFC_Working.iat[i, 11] != "Hanging Man"):
        if(HDFC_Working.iat[i, 12]/HDFC_Working.iat[i, 13] <= 0.5):
            HDFC_Working.iat[i, 11] = "Short"
        
for i in range (len(HDFC_Working)):
    
    if(HDFC_Working.iat[i, 11] != "Shooting Star" and HDFC_Working.iat[i, 11] != "Inverted Hammer" and HDFC_Working.iat[i, 11] != "Spinning Top" and HDFC_Working.iat[i, 11] != "Doji" and HDFC_Working.iat[i, 11] != "Marubozu" and HDFC_Working.iat[i, 11] != "Long" and HDFC_Working.iat[i, 11] != "Short" and HDFC_Working.iat[i, 11] != "Hammer" and HDFC_Working.iat[i, 11] != "Hanging Man"): 
        HDFC_Working.iat[i, 11] = "None"
        
HDFC_Working = HDFC_Working.drop(['CB', 'EMA'], axis = 1)

flag_1 = 0
flag_2 = 0
gap_u = 0
gap_l = 0

for i in range(len(HDFC_Working)):
    
    if(HDFC_Working.iat[i, 4] < HDFC_Working.iat[i-1, 4] < HDFC_Working.iat[i-2, 4] < HDFC_Working.iat[i-3, 4]):
        flag_1 = 1
    if(HDFC_Working.iat[i, 4] > HDFC_Working.iat[i-1, 4] > HDFC_Working.iat[i-2, 4] > HDFC_Working.iat[i-3, 4]):
        flag_2 = 1
    
    if(flag_1 == 1 and HDFC_Working.iat[i, 10]=="Bullish" and HDFC_Working.iat[i-1, 10]=="Bearish" and abs(HDFC_Working.iat[i, 6])>abs(HDFC_Working.iat[i-1, 6]) and HDFC_Working.iat[i, 4] > HDFC_Working.iat[i-1, 1] and HDFC_Working.iat[i, 1] < HDFC_Working.iat[i-1, 4]):
        HDFC_Working.iat[i, 11] = "Bullish Engulfing"
        
    if(flag_2 == 1 and HDFC_Working.iat[i, 10]=="Bearish" and HDFC_Working.iat[i-1, 10]=="Bullish" and abs(HDFC_Working.iat[i, 6])>abs(HDFC_Working.iat[i-1, 6]) and HDFC_Working.iat[i, 1] > HDFC_Working.iat[i-1, 4] and HDFC_Working.iat[i, 4] < HDFC_Working.iat[i-1, 1]):
        HDFC_Working.iat[i, 11] = "Bearish Engulfing"
        
    if(flag_1 == 1 and HDFC_Working.iat[i, 10]=="Bullish" and HDFC_Working.iat[i-1, 10]=="Bearish" and HDFC_Working.iat[i, 11]!= "Bullish Engulfing"):
        if(HDFC_Working.iat[i, 4]>HDFC_Working.iat[i-1, 1] and HDFC_Working.iat[i, 1]<HDFC_Working.iat[i-1, 1]):
            gap_u = HDFC_Working.iat[i, 4] - HDFC_Working.iat[i-1, 1]
            diff = abs(HDFC_Working.iat[i, 6]) - gap_u
            if(abs(diff)/abs(HDFC_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(HDFC_Working.iat[i-1, 6]) > 0.5):
                HDFC_Working.iat[i, 11] = "Piercing"
                
        elif(HDFC_Working.iat[i, 4]<HDFC_Working.iat[i-1, 1] and HDFC_Working.iat[i, 4]>HDFC_Working.iat[i-1, 4]):
            gap_l = HDFC_Working.iat[i-1, 4] - HDFC_Working.iat[i, 1]
            diff = abs(HDFC_Working.iat[i, 6]) - gap_l
            if(abs(diff)/abs(HDFC_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(HDFC_Working.iat[i-1, 6]) > 0.5):
                HDFC_Working.iat[i, 11] = "Piercing"        
                
                
    if(flag_2 == 1 and HDFC_Working.iat[i, 10]=="Bearish" and HDFC_Working.iat[i-1, 10]=="Bullish" and HDFC_Working.iat[i, 11]!= "Bearish Engulfing"):
        if(HDFC_Working.iat[i, 1]>HDFC_Working.iat[i-1, 4] and HDFC_Working.iat[i, 4]<HDFC_Working.iat[i-1, 4]):
            gap_u = HDFC_Working.iat[i, 1] - HDFC_Working.iat[i-1, 4]
            diff = abs(HDFC_Working.iat[i, 6]) - gap_u
            if(abs(diff)/abs(HDFC_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(HDFC_Working.iat[i-1, 6]) > 0.5):
                HDFC_Working.iat[i, 11] = "Dark Cloud Cover"
         
        elif(HDFC_Working.iat[i, 1]<HDFC_Working.iat[i-1, 4] and HDFC_Working.iat[i, 1]>HDFC_Working.iat[i-1, 1]):
            gap_l = HDFC_Working.iat[i-1, 1] - HDFC_Working.iat[i, 4]
            diff = abs(HDFC_Working.iat[i, 6]) - gap_l
            if(abs(diff)/abs(HDFC_Working.iat[i, 6]) < 0.5 and abs(diff)/abs(HDFC_Working.iat[i-1, 6]) > 0.5):
                HDFC_Working.iat[i, 11] = "Dark Cloud Cover"

    if(flag_1 == 1 and HDFC_Working.iat[i, 10]=="Bullish" and HDFC_Working.iat[i-1, 10]=="Bearish" and abs(HDFC_Working.iat[i-1, 6])>abs(HDFC_Working.iat[i, 6]) and HDFC_Working.iat[i, 4] < HDFC_Working.iat[i-1, 1] and HDFC_Working.iat[i, 1] > HDFC_Working.iat[i-1, 4]):
        HDFC_Working.iat[i, 11] = "Bullish Harami"
        
    if(flag_2 == 1 and HDFC_Working.iat[i, 10]=="Bearish" and HDFC_Working.iat[i-1, 10]=="Bullish" and abs(HDFC_Working.iat[i-1, 6])>abs(HDFC_Working.iat[i, 6]) and HDFC_Working.iat[i, 1] < HDFC_Working.iat[i-1, 4] and HDFC_Working.iat[i, 4] > HDFC_Working.iat[i-1, 1]):
        HDFC_Working.iat[i, 11] = "Bearish Harami"
        
    
for i in range(len(HDFC_Working)):
    
    if(HDFC_Working.iat[i, 10] == "Bullish" and HDFC_Working.iat[i-2, 10] == "Bearish" and HDFC_Working.iat[i-2, 4] < HDFC_Working.iat[i-3, 4] < HDFC_Working.iat[i-4, 4] and HDFC_Working.iat[i, 11] == "Long" and  (HDFC_Working.iat[i-1, 11] == "Short" or HDFC_Working.iat[i-1, 11] == "Doji" or HDFC_Working.iat[i-1, 11] == "Spinning Top") and HDFC_Working.iat[i-2, 11] == "Long"):
        HDFC_Working.iat[i, 11] = "Morning Star"
        
    if(HDFC_Working.iat[i, 10] == "Bearish" and HDFC_Working.iat[i-2, 10] == "Bullish" and HDFC_Working.iat[i-2, 4] > HDFC_Working.iat[i-3, 4] > HDFC_Working.iat[i-4, 4] and HDFC_Working.iat[i, 11] == "Long" and  (HDFC_Working.iat[i-1, 11] == "Short" or HDFC_Working.iat[i-1, 11] == "Doji" or HDFC_Working.iat[i-1, 11] == "Spinning Top") and HDFC_Working.iat[i-2, 11] == "Long"):
        HDFC_Working.iat[i, 11] = "Evening Star"
        
    if(HDFC_Working.iat[i, 4] > HDFC_Working.iat[i-1, 4] > HDFC_Working.iat[i-2, 4] and HDFC_Working.iat[i, 10] == "Bullish" and HDFC_Working.iat[i-1, 10] == "Bullish" and HDFC_Working.iat[i-2, 10] == "Bullish" and HDFC_Working.iat[i, 11] == "Long" and HDFC_Working.iat[i-1, 11] == "Long" and HDFC_Working.iat[i-2, 11] == "Long"):
        HDFC_Working.iat[i, 11] = "Soldiers"
        
    if(HDFC_Working.iat[i, 4] < HDFC_Working.iat[i-1, 4] < HDFC_Working.iat[i-2, 4] and HDFC_Working.iat[i, 10] == "Bearish" and HDFC_Working.iat[i-1, 10] == "Bearish" and HDFC_Working.iat[i-2, 10] == "Bearish" and HDFC_Working.iat[i, 10] == "Long" and HDFC_Working.iat[i-1, 11] == "Long" and HDFC_Working.iat[i-2, 11] == "Long"):
        HDFC_Working.iat[i, 11] = "Crows"
    
        
#Generating Technical Indicators

rsi_7 = RSIIndicator(HDFC_Working.Close, window = 7)
HDFC_Working["RSI_7"] = rsi_7.rsi()
        
rsi_14 = RSIIndicator(HDFC_Working.Close, window = 14)
HDFC_Working["RSI_14"] = rsi_14.rsi()  

rsi_21 = RSIIndicator(HDFC_Working.Close, window = 21)
HDFC_Working["RSI_21"] = rsi_21.rsi()

HDFC_Working["MACD (26, 12)"] = macd(HDFC_Working.Close, window_slow = 26, window_fast = 12)

HDFC_Working["MACD (44, 26)"] = macd(HDFC_Working.Close, window_slow = 44, window_fast = 26)

obv = OnBalanceVolumeIndicator(HDFC_Working.Close, HDFC_Working.Volume)
HDFC_Working["OBV"] = obv.on_balance_volume()

atr = AverageTrueRange(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Close, window = 7)
HDFC_Working["ATR_7"] = atr.average_true_range()

atr = AverageTrueRange(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Close, window = 14)
HDFC_Working["ATR_14"] = atr.average_true_range()

atr = AverageTrueRange(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Close, window = 21)
HDFC_Working["ATR_21"] = atr.average_true_range()

ao = AwesomeOscillatorIndicator(HDFC_Working.High, HDFC_Working.Low, window1 = 5, window2 = 34, fillna = False)
HDFC_Working["AO"] = ao.awesome_oscillator()

Kama = KAMAIndicator(HDFC_Working.Close, window  = 7, pow1 = 12, pow2 = 26, fillna = False)
HDFC_Working["KAMA"] = Kama.kama()

PPO = PercentagePriceOscillator(HDFC_Working.Close, window_slow = 26, window_fast = 12, window_sign = 7, fillna = False)
HDFC_Working["PPO"] = PPO.ppo()

PVO = PercentageVolumeOscillator(HDFC_Working.Volume, window_slow = 26, window_fast = 12, window_sign = 7, fillna = False)
HDFC_Working["PVO"] = PVO.pvo()

ROC = ROCIndicator(HDFC_Working.Close, window = 7, fillna = False)
HDFC_Working["ROC"] = ROC.roc()

SRSI = StochRSIIndicator(HDFC_Working.Close, window = 7, smooth1 = 12, smooth2 = 26, fillna = False)
HDFC_Working["Stochastic RSI"] = SRSI.stochrsi()

adi = AccDistIndexIndicator(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Close, HDFC_Working.Volume, fillna = False)
HDFC_Working["ADI"] = adi.acc_dist_index()

cmi = ChaikinMoneyFlowIndicator(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Close, HDFC_Working.Volume, window = 7, fillna = False)
HDFC_Working["CMI"] = cmi.chaikin_money_flow()

eom = EaseOfMovementIndicator(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Volume, window = 7, fillna = False)
HDFC_Working["EOM"] = eom.ease_of_movement() 

fi = ForceIndexIndicator(HDFC_Working.Close, HDFC_Working.Volume, window = 7, fillna = False)
HDFC_Working["FI"] = fi.force_index()

mfi = MFIIndicator(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Close, HDFC_Working.Volume, window = 7, fillna = False)
HDFC_Working["MFI"] = mfi.money_flow_index()

nvi = NegativeVolumeIndexIndicator(HDFC_Working.Close, HDFC_Working.Volume, fillna = False)
HDFC_Working["NVI"] = nvi.negative_volume_index()

vpt = VolumePriceTrendIndicator(HDFC_Working.Close, HDFC_Working.Volume, fillna = False)
HDFC_Working["VPT"] = vpt.volume_price_trend()

ADX = ADXIndicator(HDFC_Working.High, HDFC_Working.Low, HDFC_Working.Close, window = 7, fillna = False)
HDFC_Working["ADX"] = ADX.adx()

mass = MassIndex(HDFC_Working.High, HDFC_Working.Low, window_fast = 7, window_slow = 14, fillna = False)
HDFC_Working["MASS INDEX"] = mass.mass_index()

# Direction labels as per the previous day's close

HDFC_Working["Direction"] = ""

for i in range (len(HDFC_Working)):
    
    if(HDFC_Working.iat[i-1, 4] < HDFC_Working.iat[i, 4]):
        HDFC_Working.iat[i, 36] = "UP"
        
    else:
        HDFC_Working.iat[i, 36] = "DOWN"

# Target labels as per the next day's direction

HDFC_Working["Target"] = ""

for i in range (len(HDFC_Working) - 1):
    
    HDFC_Working.iat[i, 37] = HDFC_Working.iat[i+1, 36]

# Creating Training and Testing sets

HDFC_Working_Train = HDFC_Working[ : 2004]
HDFC_Working_Test = HDFC_Working[2004 : ]

HDFC_Working_Train.to_excel(r'C:\Users\MOHIT.K\Desktop\HDFC_Working_Train.xlsx', index = False)

# Dropping Date columns

HDFC_Working_Train = HDFC_Working_Train.drop(['Date'], axis =1)
HDFC_Working_Test = HDFC_Working_Test.drop(['Date'], axis =1)

# Separating textual features

HDFC_Working_Train_CN = HDFC_Working_Train.loc[ : ,['Candle Nature']]
HDFC_Working_Train_CT = HDFC_Working_Train.loc[ : ,['Candle Type']]
HDFC_Working_Train_DR = HDFC_Working_Train.loc[ : ,['Direction']]
HDFC_Working_Train_TR = HDFC_Working_Train.loc[ : ,['Target']]

# Separating numerical features 

HDFC_Working_Train_num = HDFC_Working_Train.drop(['Candle Nature', 'Candle Type', 'Direction', 'Target'], axis = 1)

#### DATA PREPROCESSING ###

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# Min Max scaling with range (0, 1)

num_pipeline_1 = Pipeline([
 ('selector_1', DataFrameSelector(list(HDFC_Working_Train_num))),
 ('scaler_1', MinMaxScaler(0, 1)),
 ])

# Min Max Scaling with range (-1, 1)

num_pipeline_2 = Pipeline([
 ('selector_2', DataFrameSelector(list(HDFC_Working_Train_num))),
 ('scaler_2', MinMaxScaler(-1, 1)),
 ])

# Encoding Candle Nature

cat_pipeline_1 = Pipeline([
 ('selector_1', DataFrameSelector(HDFC_Working_Train_CN)),
 ('label_binarizer_1', LabelBinarizer()),
 ])

#Encoding Candle Type

cat_pipeline_2 = Pipeline([
 ('selector_2', DataFrameSelector(HDFC_Working_Train_CT)),
 ('label_binarizer_2', LabelBinarizer()),
 ])

#Encoding Direction

cat_pipeline_3 = Pipeline([
 ('selector_3', DataFrameSelector(HDFC_Working_Train_DR)),
 ('label_binarizer_3', LabelBinarizer()),
 ])

#Encoding Target

cat_pipeline_4 = Pipeline([
 ('selector_4', DataFrameSelector(HDFC_Working_Train_TR)),
 ('label_binarizer_4', LabelBinarizer()),
 ])

#Creating a dataset with (0, 1) normalization

full_pipeline_1 = FeatureUnion(transformer_list=[
 ("num_pipeline_1", num_pipeline_1),
 ("cat_pipeline_1", cat_pipeline_1),
 ("cat_pipeline_2", cat_pipeline_2),
 ("cat_pipeline_3", cat_pipeline_3),
 ("cat_pipeline_4", cat_pipeline_4),
 ])

#Creating a dataset with (-1, 1) normalization

full_pipeline_2 = FeatureUnion(transformer_list=[
 ("num_pipeline_2", num_pipeline_2),
 ("cat_pipeline_1", cat_pipeline_1),
 ("cat_pipeline_2", cat_pipeline_2),
 ("cat_pipeline_3", cat_pipeline_3),
 ("cat_pipeline_4", cat_pipeline_4),
 ])

HDFC_MM1 = full_pipeline_1.fit_transform(HDFC_Working_Train)

HDFC_MM2 = full_pipeline_2.fit_transform(HDFC_Working_Train)

C:\Users\MOHIT.K\Desktop\NLP\Anaconda\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\MOHIT.K\Desktop\NLP\Anaconda\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\MOHIT.K\Desktop\NLP\Anaconda\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot
import plotly.graph_objs as go
import pandas as pd

trace = go.Candlestick(open=HDFC_X['Open'],
                high=HDFC_X['High'],
                low=HDFC_X['Low'],
                close=HDFC_X['Close'])
data = [trace]
fig = go.Figure(data=[trace])

fig.show()